In [9]:
from google.cloud import aiplatform
from kfp import dsl, compiler
from kfp.v2.dsl import component


In [10]:
PROJECT_ID = 'vertexai-457414'
REGION = 'us-central1'
BUCKET_NAME = 'gs://road_accident//'

In [11]:
PIPELINE_ROOT = f"{BUCKET_NAME}/pipelines/"

In [12]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

env: PATH=C:\Users\lpl_z\anaconda3\envs\road_accident;C:\Users\lpl_z\anaconda3\envs\road_accident\Library\mingw-w64\bin;C:\Users\lpl_z\anaconda3\envs\road_accident\Library\usr\bin;C:\Users\lpl_z\anaconda3\envs\road_accident\Library\bin;C:\Users\lpl_z\anaconda3\envs\road_accident\Scripts;C:\Users\lpl_z\anaconda3\envs\road_accident\bin;C:\Users\lpl_z\anaconda3\condabin;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0;C:\WINDOWS\System32\OpenSSH;C:\Users\lpl_z\anaconda3;C:\Users\lpl_z\anaconda3\Library\mingw-w64\bin;C:\Users\lpl_z\anaconda3\Library\usr\bin;C:\Users\lpl_z\anaconda3\Library\bin;C:\Users\lpl_z\anaconda3\Scripts;C:\Users\lpl_z\AppData\Local\Microsoft\WindowsApps:/home/jupyter/.local/bin:/home/jupyter/.local/bin


In [13]:
@dsl.component(base_image="python:3.10")
def add(a: float, b: float) -> float:
    return a + b

In [14]:
@dsl.component(base_image="python:3.10")
def multiply(a: float, b: float) -> float:
    return a * b

In [15]:
@dsl.pipeline(name="my-first-pipeline")
def add_mul_pipeline(a: float = 3, b: float = 5):
    add_task = add(a=a, b=b)
    add_task.set_caching_options(False)  # 👈 força re-execução sem cache
    _ = multiply(a=add_task.output, b=b)

compiler.Compiler().compile(
    pipeline_func=add_mul_pipeline,
    package_path="add_mul_pipeline.json",
)

In [16]:

# Inicializa o Vertex AI
aiplatform.init(project=PROJECT_ID, location=REGION)

# Cria e executa o job do pipeline
job = aiplatform.PipelineJob(
    display_name="my-second-pipeline",
    template_path="add_mul_pipeline.json",
    pipeline_root="gs://road_accident/pipelines",
    parameter_values={"a": 3, "b": 5},
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/666534829864/locations/us-central1/pipelineJobs/my-first-pipeline-20251023113818
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/666534829864/locations/us-central1/pipelineJobs/my-first-pipeline-20251023113818')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/my-first-pipeline-20251023113818?project=666534829864
PipelineJob projects/666534829864/locations/us-central1/pipelineJobs/my-first-pipeline-20251023113818 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/666534829864/locations/us-central1/pipelineJobs/my-first-pipeline-20251023113818 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/666534829864/locations/us-central1/pipelineJobs/my-first-pipeline-20251023113818 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/666534829864/locations/us-central1/pip